In [1]:
!pip install -q langchain numexpr sentencepiece plotly diffusers


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from langchain.agents import load_tools
from langchain.chat_models import ChatHuggingFace, ChatOpenAI
from langchain_core.tools import Tool
import numpy as np
import pandas as pd
from dotenv import load_dotenv

load_dotenv(override=True)

from scripts.modified_calculator import LLMMathChain
from scripts.run_agents import answer_questions

In [4]:
OPENAI_MODEL_ID = "gpt-4-0125-preview"

In [5]:
from scripts.evaluation import load_benchmark
import datasets

eval_df = load_benchmark()
eval_df["true_answer"] = eval_df["true_answer"].astype(str)
eval_ds = datasets.Dataset.from_pandas(eval_df)

OUTPUT_DIR = "output"

In [6]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    max_new_tokens=3000,
)
llm_engine = ChatHuggingFace(llm=llm)


def call_llm(input: str, stop=["Observation", "Final Answer"]) -> str:
    return llm_engine.invoke(input, stop=stop).content


call_llm("I should multiply 2 by 3.6452.")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


' To multiply 2 by the number 3.6452, you can simply multiply each digit of 2 by each digit of 3.6452 and sum up the results. However, since 2 is a single digit number, you can just multiply it by each digit of 3.6452 and sum up the results:\n\n2 \\* 3 = 6\n2 \\* 6 = 12 (remember to carry the "1" to the next digit)\n2 \\* 4 = 8 (add the carried "1" to this digit, making it 9)\n2 \\* 5 = 10 (add the carried "1" to this digit, making it 11, but since we only have one digit after the decimal point, we need to round it to the nearest whole number, so the final result is 7.6452, but since we only want to keep 4 decimal places, we round it to 7.6452)\n\nTherefore, 2 \\* 3.6452 = 7.6452.'

### Define tools

In [7]:
from transformers.tools.base import CalculatorTool, Tool

from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "bing",
    "gl": "us",
    "hl": "en",
}
langchain_serpapi = SerpAPIWrapper(params=params)


class SearchTool(Tool):
    name = "search"
    description = "Search the web for information."

    inputs = {"query": str}
    outputs = [str]

    def __call__(self, query: str) -> str:
        return langchain_serpapi.run(query)

In [8]:
search = SearchTool()

In [9]:
TOOLBOX = [CalculatorTool(), SearchTool()]

# Define agents

In [45]:
from transformers import CodeAgent, ReactAgent

# code_agent = CodeAgent(llm_callable=call_llm, toolbox=TOOLBOX)
# react_agent = ReactAgent(llm_callable=call_llm, toolbox=TOOLBOX)

# agents = {
#     "code": code_agent,
#     "react": react_agent,
# }

agents = {
    "vanilla_llm": llm,
}

In [46]:
async def call_transformers(agent, question: str) -> str:
    result = agent.run(question)
    return {
        "output": str(result),
        "intermediate_steps": agent.messages.copy(),
    }


async def call_llm(agent, question: str) -> str:
    result = agent(question)
    return {
        "output": str(result),
        "intermediate_steps": [],
    }

In [44]:
await call_llm(llm, "I should multiply 2 by 3.6452.")

{'output': '\nAnswer: 7.2904', 'intermediate_steps': []}

In [47]:
from scripts.run_agents import run_full_tests

await run_full_tests(
    eval_ds,
    agents,
    output_folder=OUTPUT_DIR,
    agent_call_function=call_llm,
    key_for_answer="true_answer",
)

100%|██████████| 90/90 [23:22<00:00, 15.58s/it] 


,agent_name,question,prediction,intermediate_steps,parsing_error,iteration_limit_exceeded,agent_error,start_time,end_time,gt_answer,task
0,vanilla_llm,Mimi picked up 2 dozen seashells on the beach....,\n\nLet's reason through this problem:\n\n1. M...,[],False,False,None,2024-03-05 18:44:46,2024-03-05 18:44:51,16.0,gsm8k
1,vanilla_llm,Frankie's parents let him have many pets. He h...,"\n\nFirst, we need to find out how many cats F...",[],False,False,None,2024-03-05 18:44:51,2024-03-05 18:45:12,19.0,gsm8k
2,vanilla_llm,"Olaf collects colorful toy cars. At first, his...","\n\nLet's break this problem down:\n1. First, ...",[],False,False,None,2024-03-05 18:45:12,2024-03-05 18:45:20,196.0,gsm8k
3,vanilla_llm,Emma's bank account has $100 in it. Each day o...,\n\nHere's how to tackle this problem:\n\n1. E...,[],False,False,None,2024-03-05 18:45:20,2024-03-05 18:45:25,4.0,gsm8k
4,vanilla_llm,"Ezekiel hikes as a hobby. This past summer, he...",\n\n### Watch This\n\nFirst watch this video.\...,[],False,False,None,2024-03-05 18:45:25,2024-03-05 18:46:49,15.0,gsm8k
...,...,...,...,...,...,...,...,...,...,...,...
85,vanilla_llm,How many at bats did the Yankee with the most ...,\n\nReggie Jackson had the most walks in the 1...,[],False,False,None,2024-03-05 19:05:54,2024-03-05 19:06:01,519,GAIA
86,vanilla_llm,In Audre Lorde’s poem “Father Son and Holy Gho...,\n\n1. Second\n2. Third\n3. Fourth\n4. Fifth\n...,[],False,False,None,2024-03-05 19:06:01,2024-03-05 19:06:03,2,GAIA
87,vanilla_llm,Where were the Vietnamese specimens described ...,\n\nTonkin (northern Vietnam)\n\nHaiphong\n\n-...,[],False,False,None,2024-03-05 19:06:03,2024-03-05 19:07:37,Saint Petersburg,GAIA
88,vanilla_llm,What country had the least number of athletes ...,\n\nAnswer: MON (Monaco)\n\nMonaco is a small ...,[],False,False,None,2024-03-05 19:07:37,2024-03-05 19:07:44,CUB,GAIA


# Evaluate

In [48]:
import glob

answer_file_path = f"{OUTPUT_DIR}/answers.jsonl"

result_df = pd.concat([pd.read_json(f) for f in glob.glob(f"{OUTPUT_DIR}/*.json")])
result_df = result_df.drop(columns=["start_time", "end_time"])
result_df.to_json(answer_file_path, lines=True, orient="records")

### Exact match for GSM8K

In [49]:
from scripts.evaluation import score_any_match

results_math = result_df.loc[result_df["task"] == "gsm8k"].copy()
results_math["exact_match"] = -1
results_math["exact_match"] = results_math.apply(
    lambda row: score_any_match(row["prediction"], float(row["gt_answer"])), axis=1
)

Error when extracting string: could not convert string to float: '2U'
Error when extracting string: could not convert string to float: '4x'
Error when extracting string: could not convert string to float: '3r'


In [50]:
results_math["agent_name"].value_counts()

agent_name
vanilla_llm    40
code           40
react          40
react_prev     40
Name: count, dtype: int64

In [51]:
results_math.groupby(["agent_name", "task"])["exact_match"].mean()

agent_name   task 
code         gsm8k    0.675
react        gsm8k    0.375
react_prev   gsm8k    0.425
vanilla_llm  gsm8k    0.650
Name: exact_match, dtype: float64

### LLM as a judge for others

In [52]:
answers = result_df.loc[result_df["task"] != "gsm8k"].to_dict(orient="records")

In [53]:
from langchain.chat_models import ChatOpenAI

eval_chat_model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
eval_model_name = "GPT4"

In [54]:
from scripts.evaluation import evaluate_answers
from scripts.prompts import EVALUATION_PROMPT_TEMPLATE

output_file_path = f"{OUTPUT_DIR}/evaluation.jsonl"

run_evaluation = True
if run_evaluation:
    results = await evaluate_answers(
        answers,
        eval_chat_model,
        "GPT4",
        EVALUATION_PROMPT_TEMPLATE,
        output_file_path=output_file_path,
    )

    print("Evaluation is complete!")

Previous evaluations:
Launching evaluation for 50 examples...


  0%|          | 0/50 [00:00<?, ?it/s]

Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example
Evaluating example


100%|██████████| 50/50 [00:26<00:00,  1.89it/s]

Evaluation is complete!


In [55]:
results_nonmath = pd.DataFrame.from_dict(results)


def interpret_score(eval_score):
    try:
        return (float(eval_score) - 1) / 4
    except:
        return 0


results_nonmath["eval_score_GPT4"] = results_nonmath["eval_score_GPT4"].apply(
    interpret_score
)
results_nonmath.groupby("agent_name")["eval_score_GPT4"].mean()

agent_name
code           0.255
react          0.345
react_prev     0.335
vanilla_llm    0.440
Name: eval_score_GPT4, dtype: float64

### Aggregate both

In [61]:
result_df = pd.concat([results_math, results_nonmath])
result_df["aggregate_score"] = (
    result_df["exact_match"].fillna(0) + result_df["eval_score_GPT4"].fillna(0)
) * 100
result_df.groupby(["agent_name", "task"])["aggregate_score"].mean()

agent_name   task    
code         GAIA             12.5
             HotpotQA    34.166667
             gsm8k            67.5
react        GAIA            13.75
             HotpotQA    48.333333
             gsm8k            37.5
react_prev   GAIA             8.75
             HotpotQA         50.0
             gsm8k            42.5
vanilla_llm  GAIA            26.25
             HotpotQA    55.833333
             gsm8k            65.0
Name: aggregate_score, dtype: object

### Display

In [ ]:
result_df

In [67]:
import plotly.express as px

aggregate = (
    result_df.groupby(["agent_name", "task"])[["aggregate_score"]].mean().reset_index()
)
aggregate["agent_name"] = aggregate["agent_name"].map(
    {
        "vanilla_llm": "LLM",
        "react": "ReactAgent",
        "code": "CodeAgent",
        "react_prev": "ReactAgent (previous)",
    }
)
fig = px.bar(
    aggregate,
    x="agent_name",
    y="aggregate_score",
    color="task",
    labels={
        "agent_name": "<b>Agent</b>",
        "task": "<b>Task</b>",
        "aggregate_score": "Performance",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    width=len(aggregate["agent_name"].unique()) * 200 + 200,
    height=600,
    barmode="group",
    bargap=0.35,
    bargroupgap=0.0,
    yaxis_range=[0, 100],
)
fig.update_traces(texttemplate="%{y:.0f}", textposition="outside")
fig.layout.yaxis.ticksuffix = "%"
fig.show()

/Users/aymeric/venvs/ml/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

